# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


## Preview Data

In [1]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
print("df shape:"), print(df.shape), print("---"*20)
print("df isna().sum().sort_values(ascending=False):"), print(df.isna().sum().sort_values(ascending=False)), print("---"*20)

df shape:
(7043, 21)
------------------------------------------------------------
df isna().sum().sort_values(ascending=False):
Churn               0
OnlineSecurity      0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineBackup        0
TotalCharges        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
customerID          0
dtype: int64
------------------------------------------------------------


(None, None, None)

## Encode and Standardize Data

In [8]:
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

encoder = ce.OrdinalEncoder()
df = encoder.fit_transform(df)

X = df.drop('Churn', axis=1)
y = df['Churn']

# Scale input data
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

## Create and fit a baseline Keras MLP model to the data.

In [11]:
from keras import optimizers
from keras.layers import Dense, Dropout
from keras.models import Sequential


# Hyperparameters
inputs = X.shape[1]
epochs = 20
batch_size = 80

# Create model
model = Sequential()
model.add(Dense(24, activation='relu', input_shape=(inputs,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Fit model
model.summary()
model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 24)                504       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                400       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 921
Trainable params: 921
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 5634 samples, validate on 1409 samples
Epoch 1/20
5634/5634 [==============================] - 3s 542us/step - loss: 0.7920 - acc: 0.4815 - val_loss: 0.3812 - val_acc: 0.6955
Epoch 2/20
5634/5634 [==============================] - 0s 83us/step - loss: 0.2842 - acc: 0.7300 - val_loss: 0.2704 - val_acc: 0.7296
Epoch 3/20
5634/5634 [===================

## Hyperparameter tune (at least) the following parameters:

* batch_size
* training epochs
* optimizer
* learning rate (if applicable to optimizer)
* momentum (if applicable to optimizer)
* activation functions
* network weight initialization
* dropout regularization
* number of neurons in the hidden layer
* You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters

In [20]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=(inputs,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define cross validation test harness - n_splits=5-10 to ensure accuracy
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Define the grid search parameters
param_grid = {'batch_size': [20, 30, 40, 80],
              'epochs': [20, 40]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=kfold)
grid_result = grid.fit(X, y)

Epoch 1/20
4695/4695 [==============================] - 6s 1ms/step - loss: 4.9392 - acc: 0.6709
Epoch 2/20
4695/4695 [==============================] - 2s 466us/step - loss: 4.1426 - acc: 0.7361
Epoch 3/20
4695/4695 [==============================] - 2s 508us/step - loss: 7.4533 - acc: 0.5050
Epoch 4/20
4695/4695 [==============================] - 2s 458us/step - loss: 5.3174 - acc: 0.6603
Epoch 5/20
4695/4695 [==============================] - 2s 478us/step - loss: 4.0936 - acc: 0.7297
Epoch 6/20
4695/4695 [==============================] - 2s 498us/step - loss: 3.9015 - acc: 0.7318
Epoch 7/20
4695/4695 [==============================] - 3s 548us/step - loss: 3.9664 - acc: 0.7350
Epoch 8/20
4695/4695 [==============================] - 2s 529us/step - loss: 3.9025 - acc: 0.7289
Epoch 9/20
4695/4695 [==============================] - 2s 430us/step - loss: 4.0633 - acc: 0.7335
Epoch 10/20
4695/4695 [==============================] - 2s 430us/step - loss: 3.8718 - acc: 0.7359
Epoch 11/20

4695/4695 [==============================] - 1s 284us/step - loss: 0.7821 - acc: 0.7410
Epoch 20/40
4695/4695 [==============================] - 1s 256us/step - loss: 0.5922 - acc: 0.7534
Epoch 21/40
4695/4695 [==============================] - 1s 317us/step - loss: 0.5954 - acc: 0.7551
Epoch 22/40
4695/4695 [==============================] - 1s 319us/step - loss: 0.6950 - acc: 0.7427
Epoch 23/40
4695/4695 [==============================] - 2s 394us/step - loss: 0.6201 - acc: 0.7470
Epoch 24/40
4695/4695 [==============================] - 2s 480us/step - loss: 0.5816 - acc: 0.7474
Epoch 25/40
4695/4695 [==============================] - 3s 538us/step - loss: 0.5656 - acc: 0.7591
Epoch 26/40
4695/4695 [==============================] - 3s 586us/step - loss: 0.5530 - acc: 0.7649
Epoch 27/40
4695/4695 [==============================] - 2s 381us/step - loss: 0.6471 - acc: 0.7438
Epoch 28/40
4695/4695 [==============================] - 2s 345us/step - loss: 0.6656 - acc: 0.7410
Epoch 29/40


4696/4696 [==============================] - 2s 333us/step - loss: 4.0426 - acc: 0.7321
Epoch 19/40
4696/4696 [==============================] - 2s 345us/step - loss: 3.8473 - acc: 0.7313
Epoch 20/40
4696/4696 [==============================] - 2s 458us/step - loss: 3.5253 - acc: 0.7347
Epoch 21/40
4696/4696 [==============================] - 2s 516us/step - loss: 3.5870 - acc: 0.7408
Epoch 22/40
4696/4696 [==============================] - 2s 454us/step - loss: 3.5906 - acc: 0.7349
Epoch 23/40
4696/4696 [==============================] - 2s 466us/step - loss: 3.6735 - acc: 0.7332
Epoch 24/40
4696/4696 [==============================] - 2s 476us/step - loss: 3.4797 - acc: 0.7298
Epoch 25/40
4696/4696 [==============================] - 2s 470us/step - loss: 3.7205 - acc: 0.7353
Epoch 26/40
4696/4696 [==============================] - 2s 473us/step - loss: 4.0981 - acc: 0.7357
Epoch 27/40
4696/4696 [==============================] - 2s 462us/step - loss: 3.7959 - acc: 0.7221
Epoch 28/40


4696/4696 [==============================] - 1s 227us/step - loss: 2.7494 - acc: 0.7121
Epoch 17/20
4696/4696 [==============================] - 1s 267us/step - loss: 3.9097 - acc: 0.7347
Epoch 18/20
4696/4696 [==============================] - 2s 355us/step - loss: 2.8795 - acc: 0.7247
Epoch 19/20
4696/4696 [==============================] - 2s 509us/step - loss: 3.0672 - acc: 0.7236
Epoch 20/20
4696/4696 [==============================] - 1s 137us/step
Epoch 1/40
4695/4695 [==============================] - 9s 2ms/step - loss: 4.1097 - acc: 0.6990
Epoch 2/40
4695/4695 [==============================] - 1s 318us/step - loss: 3.8903 - acc: 0.7248
Epoch 3/40
4695/4695 [==============================] - 2s 466us/step - loss: 2.1321 - acc: 0.7033
Epoch 4/40
4695/4695 [==============================] - 2s 410us/step - loss: 1.4059 - acc: 0.7212
Epoch 5/40
4695/4695 [==============================] - 1s 203us/step - loss: 1.2795 - acc: 0.7188
Epoch 6/40
4695/4695 [==========================

4695/4695 [==============================] - 1s 210us/step - loss: 0.6181 - acc: 0.7602
Epoch 36/40
4695/4695 [==============================] - 1s 157us/step - loss: 0.8000 - acc: 0.7282
Epoch 37/40
4695/4695 [==============================] - 1s 151us/step - loss: 0.6909 - acc: 0.7465
Epoch 38/40
4695/4695 [==============================] - 1s 150us/step - loss: 0.7849 - acc: 0.7376
Epoch 39/40
4695/4695 [==============================] - 1s 148us/step - loss: 0.6233 - acc: 0.7585
Epoch 40/40
4695/4695 [==============================] - 0s 75us/step
Epoch 1/40
4696/4696 [==============================] - 6s 1ms/step - loss: 4.2745 - acc: 0.7347
Epoch 2/40
4696/4696 [==============================] - 1s 271us/step - loss: 4.1760 - acc: 0.7279
Epoch 3/40
4696/4696 [==============================] - 1s 208us/step - loss: 3.7452 - acc: 0.7078
Epoch 4/40
4696/4696 [==============================] - 1s 180us/step - loss: 1.7709 - acc: 0.7006
Epoch 5/40
4696/4696 [==========================

4695/4695 [==============================] - 1s 124us/step - loss: 4.2776 - acc: 0.7346
Epoch 14/20
4695/4695 [==============================] - 1s 121us/step - loss: 4.2776 - acc: 0.7346
Epoch 15/20
4695/4695 [==============================] - 1s 156us/step - loss: 4.2776 - acc: 0.7346
Epoch 16/20
4695/4695 [==============================] - 1s 149us/step - loss: 4.2776 - acc: 0.7346
Epoch 17/20
4695/4695 [==============================] - 1s 126us/step - loss: 4.2776 - acc: 0.7346
Epoch 18/20
4695/4695 [==============================] - 1s 120us/step - loss: 4.2776 - acc: 0.7346
Epoch 19/20
4695/4695 [==============================] - 1s 118us/step - loss: 4.2776 - acc: 0.7346
Epoch 20/20
4695/4695 [==============================] - 1s 122us/step
Epoch 1/20
4696/4696 [==============================] - 6s 1ms/step - loss: 4.1613 - acc: 0.7176
Epoch 2/20
4696/4696 [==============================] - 1s 132us/step - loss: 4.0610 - acc: 0.7266
Epoch 3/20
4696/4696 [=======================

4695/4695 [==============================] - 1s 145us/step - loss: 4.6730 - acc: 0.6745
Epoch 12/40
4695/4695 [==============================] - 1s 134us/step - loss: 4.1639 - acc: 0.7274
Epoch 13/40
4695/4695 [==============================] - 1s 120us/step - loss: 3.9464 - acc: 0.7255
Epoch 14/40
4695/4695 [==============================] - 1s 143us/step - loss: 4.0989 - acc: 0.7353
Epoch 15/40
4695/4695 [==============================] - 1s 126us/step - loss: 3.5733 - acc: 0.7259
Epoch 16/40
4695/4695 [==============================] - 1s 139us/step - loss: 4.1825 - acc: 0.7346
Epoch 17/40
4695/4695 [==============================] - 1s 160us/step - loss: 4.1195 - acc: 0.7346
Epoch 18/40
4695/4695 [==============================] - 1s 119us/step - loss: 3.9204 - acc: 0.7214
Epoch 19/40
4695/4695 [==============================] - 1s 129us/step - loss: 4.0042 - acc: 0.7344
Epoch 20/40
4695/4695 [==============================] - 1s 122us/step - loss: 4.1191 - acc: 0.7350
Epoch 21/40


4695/4695 [==============================] - 0s 69us/step - loss: 2.7755 - acc: 0.7220
Epoch 12/20
4695/4695 [==============================] - 0s 65us/step - loss: 2.1677 - acc: 0.7125
Epoch 13/20
4695/4695 [==============================] - 0s 76us/step - loss: 1.7394 - acc: 0.7280
Epoch 14/20
4695/4695 [==============================] - 0s 76us/step - loss: 1.3699 - acc: 0.7329
Epoch 15/20
4695/4695 [==============================] - 0s 100us/step - loss: 1.3797 - acc: 0.7218
Epoch 16/20
4695/4695 [==============================] - 0s 84us/step - loss: 1.1833 - acc: 0.7335
Epoch 17/20
4695/4695 [==============================] - 0s 69us/step - loss: 2.5721 - acc: 0.7252
Epoch 18/20
4695/4695 [==============================] - 0s 75us/step - loss: 1.7199 - acc: 0.7212
Epoch 19/20
4695/4695 [==============================] - 0s 86us/step - loss: 1.0174 - acc: 0.7344
Epoch 20/20
4695/4695 [==============================] - 0s 59us/step
Epoch 1/20
4695/4695 [============================

4695/4695 [==============================] - 0s 73us/step - loss: 0.6741 - acc: 0.7391
Epoch 31/40
4695/4695 [==============================] - 1s 108us/step - loss: 0.7647 - acc: 0.7291
Epoch 32/40
4695/4695 [==============================] - 1s 115us/step - loss: 0.6124 - acc: 0.7521
Epoch 33/40
4695/4695 [==============================] - 0s 65us/step - loss: 0.5452 - acc: 0.7602
Epoch 34/40
4695/4695 [==============================] - 0s 65us/step - loss: 0.5715 - acc: 0.7570
Epoch 35/40
4695/4695 [==============================] - 0s 64us/step - loss: 0.5137 - acc: 0.7651
Epoch 36/40
4695/4695 [==============================] - 0s 75us/step - loss: 0.6796 - acc: 0.7346
Epoch 37/40
4695/4695 [==============================] - 0s 76us/step - loss: 0.5699 - acc: 0.7604
Epoch 38/40
4695/4695 [==============================] - 0s 67us/step - loss: 0.6712 - acc: 0.7425
Epoch 39/40
4695/4695 [==============================] - 0s 67us/step - loss: 0.6916 - acc: 0.7378
Epoch 40/40
4695/469

4696/4696 [==============================] - 0s 78us/step - loss: 2.8519 - acc: 0.7362
Epoch 31/40
4696/4696 [==============================] - 0s 75us/step - loss: 3.6520 - acc: 0.7355
Epoch 32/40
4696/4696 [==============================] - 0s 68us/step - loss: 2.9455 - acc: 0.7285
Epoch 33/40
4696/4696 [==============================] - 0s 69us/step - loss: 2.9798 - acc: 0.7359
Epoch 34/40
4696/4696 [==============================] - 0s 94us/step - loss: 2.4983 - acc: 0.7411
Epoch 35/40
4696/4696 [==============================] - 0s 105us/step - loss: 3.3582 - acc: 0.7332
Epoch 36/40
4696/4696 [==============================] - 0s 100us/step - loss: 2.3522 - acc: 0.7342
Epoch 37/40
4696/4696 [==============================] - 0s 103us/step - loss: 2.9717 - acc: 0.7223
Epoch 38/40
4696/4696 [==============================] - 0s 102us/step - loss: 4.1045 - acc: 0.7351
Epoch 39/40
4696/4696 [==============================] - 0s 100us/step - loss: 3.2593 - acc: 0.7359
Epoch 40/40
4696/

## Result

In [23]:
# Report Results
print(f"Best score: {grid_result.best_score_} , best_params: {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

print("-"*100)
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best score: 0.7688485015594669 , best_params: {'batch_size': 80, 'epochs': 40}
----------------------------------------------------------------------------------------------------
Means: 0.7364759336272102, Stdev: 0.0035720605542494685 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7471248040344166, Stdev: 0.013956396344378738 with: {'batch_size': 20, 'epochs': 40}
Means: 0.7387476939547727, Stdev: 0.004098768799734154 with: {'batch_size': 30, 'epochs': 20}
Means: 0.7434331944478636, Stdev: 0.06852275431985505 with: {'batch_size': 30, 'epochs': 40}
Means: 0.734204174839907, Stdev: 0.0005775310301765439 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7560698578561823, Stdev: 0.018459236252544208 with: {'batch_size': 40, 'epochs': 40}
Means: 0.7606133787651961, Stdev: 0.017881088888334434 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7688485015594669, Stdev: 0.03450246986040587 with: {'batch_size': 80, 'epochs': 40}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?